In [16]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
import math

In [17]:
data = pd.read_csv(r'C:\Users\Asus\PycharmProjects\FPL Project\my_data\final_dataset.csv')

In [20]:
data.isnull().values.any()

False

In [19]:
data = data.dropna()

In [14]:
# data.info(verbose=True)

**dropping gks to imporve model accuracy by removing some feature not related to outfield players**

In [21]:
data_wo_gk = data.drop(data[(data['position_name'] == 'GK')].index)

In [22]:
# data_refined= data_wo_gk.drop(['kickoff_time', 'position', 'penalties_saved', 'saves', 'transfers_balance'], axis = 1)
data_refined = data_wo_gk.loc[:,[i for i in list(data_wo_gk.columns) if i not in ['player','kickoff_time', 'position',
                                                                      'penalties_saved', 'saves', 'transfers_balance']]]

In [13]:
# data_refined.info(verbose=True)

In [23]:
#one-hot encoding for player position
data_refined = pd.get_dummies(data_refined, columns = ['position_name', 'gw', 'season', 'team','opponent_team','was_home'], drop_first=True)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(data_refined.loc[:, data_refined.columns != 'total_points'], data_refined['total_points'], test_size=0.2, random_state=0)

In [25]:
#dict to store results:
results_train = {}
results_test = {}

In [28]:
#Random Forest
reg = RandomForestRegressor(random_state=0, criterion = 'mse')
#Apply grid search for best parameters
params = {'randomforestregressor__n_estimators' : range(100, 500, 200),
          'randomforestregressor__min_samples_split' : range(2, 10, 3)}
pipe = make_pipeline(reg)
grid = GridSearchCV(pipe, param_grid = params, n_jobs=-1, iid=False, cv=5)
reg = grid.fit(X_train, y_train)
print('Best MSE: ', grid.best_score_)
print('Best Parameters: ', grid.best_estimator_)

y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)
tr_err = mean_squared_error(y_train_pred, y_train)
ts_err = mean_squared_error(y_test_pred, y_test)
print(tr_err, ts_err)
results_train['random_forest'] = tr_err
results_test['random_forest'] = ts_err

C:\Users\Asus\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


Best MSE:  -0.048342588025416786
Best Parameters:  Pipeline(steps=[('randomforestregressor',
                 RandomForestRegressor(n_estimators=300, random_state=0))])
0.005641808969442835 0.03204581935616761


In [148]:
reg = GradientBoostingRegressor(random_state=0, n_estimators = 100, min_samples_split = 5).fit(X_train, y_train)
y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)
tr_err = mean_squared_error(y_train_pred, y_train)
ts_err = mean_squared_error(y_test_pred, y_test)
print(tr_err, ts_err)

0.11350665895052597 0.1132750362307757


In [149]:
#Lasso Linear Regression
reg = linear_model.Lasso()
#Apply grid search for best parameters
params = {'lasso__alpha':[0.001, 0.01, 0.1, 0.5, 0.9]}
pipe = make_pipeline(reg)
grid = GridSearchCV(pipe, param_grid = params, scoring='mean_squared_error', n_jobs=-1, iid=False, cv=5)
reg = grid.fit(X_train, y_train)
print('Best MSE: ', grid.best_score_)
print('Best Parameters: ', grid.best_estimator_)

ValueError: 'mean_squared_error' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [151]:
print(sorted(sklearn.metrics.SCORERS.keys()))

NameError: name 'sklearn' is not defined

In [ ]:
#Ridge Linear Regression
reg = Ridge()
#Apply grid search for best parameters

params = {'ridge__alpha':[0.0001, 0.01, 0.1, 0.5, 0.9],
          'ridge__solver':['svd', 'cholesky', 'lsqr', 'sag']}
pipe = make_pipeline(reg)
grid = GridSearchCV(pipe, param_grid = params, scoring='mean_squared_error', n_jobs=-1, iid=False, cv=5)
reg = grid.fit(X_train, y_train)
print('Best MSE: ', grid.best_score_)
print('Best Parameters: ', grid.best_estimator_)

y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)
tr_err = mean_squared_error(y_train_pred, y_train)
ts_err = mean_squared_error(y_test_pred, y_test)
print(tr_err, ts_err)
results_train['Ridge_regression'] = tr_err
results_test['Ridge_regression'] = ts_err